<a href="https://colab.research.google.com/github/solanods/ProjetosFaculdade/blob/main/NLP_with_Spacy_and_Sklearn_refactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Identificação**: Solano Cruz Júnior (2919585), Rogério Dourado Felix (2892220), Newton Douglas da Silva Nascimento (2733408), Raimundo de Sousa Lima Filho (2709691)

---





# Tema: Natural Language Processing (NPL) para análise de sentimentos em postagens do Twitter

O uso de inteligência artificial e machine learning para atividades de mineração de textos da web se apresenta como uma riquíssima fonte de insights para a ciência de dados. Nesse contexto, frameworks de aprendizado de máquina como Sklearn, Keras e Tensorflow têm dominado a cena, realizando desde tarefas simples até a criação de autoresponders, bots, classificadores de texto e vários sistemas de diálogo que determinam o significado das frases. 

Um framework bastante poderoso e com uma curva de aprendizagem um pouco mais curta em relação aos já citados é a biblioteca Spacy. Escrita em Python, possui suporte para mais de 60 idiomas. De código aberto, foi projetada para construir sistemas de extração de informações ou de compreensão de linguagem natural. Também foi desenvolvida para uso em produção e fornece uma API concisa e fácil de usar.


> **Objetivo**

Esse projeto tem como objetivo treinar um modelo capaz de avaliar se uma determinada frase apresenta sentimento POSITIVO ou NEGATIVO. 

> **Especificação Técnica**


Dataset: Para desenvolvimento desse projeto, será utilizado o dataset
denominado Train50, disponível em:  https://github.com/solanods/ProjetosFaculdade/blob/main/Train50.csv.



> **Formato:** A base de dado está em formato CSV. Train 50.csv possui 50mil registros e 5 colunas. 

> **Métodos de Pŕe-processamento:** Será criada uma função em Python para a limpeza e pré-processamentos textuais

Letras minúsculas: serão priorizadas no case das strings.<br>
Nome do usuário: será removido junto do caractere (@)<br>
URLs: remoção<br>
Espaços em branco: remoção<br>
Stop words: remoção<br>
Lematização: aplicação<br>
Pontuações: remoção<br>



> **Tarefa de Aprendizado:** 

Serão utilizados métodos e funções do Sklearn a saber:



TfidfVectorizer: Converte uma coleção de documentos de texto em uma matriz de contagens de token

accuracy_score: calcula a precisão do subconjunto 


Pipeline:  usado para encadear vários estimadores em um. Isso é útil, pois geralmente há uma sequência fixa de etapas no processamento dos dados, por exemplo, seleção de recursos, normalização e classificação.

LinearSVC: classes capaz de realizar classificação binária e multiclasse em um conjunto de dados.





 








IMPORTAÇÃO E INSTALAÇÃO DAS BIBLIOTECAS

In [1]:
!pip install -q spacy==2.2.3 

     |████████████████████████████████| 10.4 MB 5.1 MB/s 
     |████████████████████████████████| 2.2 MB 46.2 MB/s 


In [2]:
#atualização da biblioteca Spacy para português
!python3 -m spacy download pt


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 21.2 MB 1.2 MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-py3-none-any.whl size=21186281 sha256=3338c9acae84f2ec549ee25911b03e36c28bd51cadfbcb82c45627d08aa12f64
  Stored in directory: /tmp/pip-ephem-wheel-cache-5c2gyl24/wheels/c3/f9/0c/5c014a36941a00f5df5fc0756cb961d7c457a978e697a6ce3b
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/pt_core_news_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/pt
You can now load the model via spacy.load('pt')


In [3]:
import pandas as pd
import string
import spacy
import re
# Machine learning imports
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score 
from sklearn.pipeline import Pipeline
from sklearn.svm import LinearSVC
# Divide o dataset em treino e teste
from sklearn.model_selection import train_test_split

In [4]:
#carregar os dados
df = pd.read_csv('/content/Train50.csv', delimiter=';')

In [5]:
df.shape

(50000, 5)

In [6]:
df.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1050785521201541121,@Laranjito76 A pessoa certa para isso seria o ...,Fri Oct 12 16:29:25 +0000 2018,1,:)
1,1050785431955140608,"@behin_d_curtain Para mim, é precisamente o co...",Fri Oct 12 16:29:04 +0000 2018,1,:)
2,1050785401248645120,Vou fazer um video hoje... estou pensando em f...,Fri Oct 12 16:28:56 +0000 2018,1,:)
3,1050785370982547461,"aaaaaaaa amei tanto essas polaroids, nem sei e...",Fri Oct 12 16:28:49 +0000 2018,1,:)
4,1050785368902131713,Valoriza o coração do menininho que vc tem. El...,Fri Oct 12 16:28:49 +0000 2018,1,:)


In [7]:
#base de dados com 5000 registros (tweets)
#sentimento negativo label 0
#sentimento postivo label 1

df['sentiment'].value_counts()

1    25000
0    25000
Name: sentiment, dtype: int64

In [8]:
# o conjunto de dados possui duas colunas do tipo inteiro e três colunas do tipo string
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   id          50000 non-null  int64 
 1   tweet_text  50000 non-null  object
 2   tweet_date  50000 non-null  object
 3   sentiment   50000 non-null  int64 
 4   query_used  50000 non-null  object
dtypes: int64(2), object(3)
memory usage: 1.9+ MB


In [9]:
#eliminamos as colunas que não vamos usar
df.drop(['id', 'tweet_date', 'query_used'], axis = 1, inplace=True)

In [10]:
df.head()

,tweet_text,sentiment
0,@Laranjito76 A pessoa certa para isso seria o ...,1
1,"@behin_d_curtain Para mim, é precisamente o co...",1
2,Vou fazer um video hoje... estou pensando em f...,1
3,"aaaaaaaa amei tanto essas polaroids, nem sei e...",1
4,Valoriza o coração do menininho que vc tem. El...,1


In [11]:
#confirmar se não há valores nulos no dataframe
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_text  50000 non-null  object
 1   sentiment   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


# Função para pré-processamento dos textos

## Serão eliminados de cada registro da base de dados

###Letras maiúsculas
###Nome do usuário (@)
###URLs
###Espaços em branco
###Stop words
###Lematização
###Pontuações

In [12]:
# carregamento  do objeto spacy em português
nlp = spacy.load('pt')
nlp

In [13]:
# construir uma lista de stop words a serem filtradas
stop_words = spacy.lang.pt.stop_words.STOP_WORDS

Stop words (ou palavras de parada – tradução livre) são palavras que podem ser consideradas irrelevantes para o conjunto de resultados a ser exibido em uma busca realizada em uma search engine. Exemplos: as, e, os, de, para, com, sem, foi.

In [14]:
def preprocessamento(texto):
  # manter tudo em letras minúsculas
  texto = texto.lower()

  # eliminar nome do usuário
  texto = re.sub(r"@[A-Za-z0-9$-_@.&+]+", ' ', texto)

  # eliminar urls
  texto = re.sub(r"https?://[A-Za-z0-9./]+", ' ', texto)

  # eliminar espaços em branco
  texto = re.sub(r" +", ' ', texto)

   # Lematização
  documento = nlp(texto)

  lista = []
  for token in documento:
    lista.append(token.lemma_)
  
  # Usamos uma compressão de lista para eliminar Stop words e pontuações
  lista = [palavra for palavra in lista if palavra not in stop_words and palavra not in string.punctuation]
  lista = ' '.join([str(elemento) for elemento in lista if not elemento.isdigit()])
  
  return lista



## AGORA VAMOS APLICAR NOSSA FUNÇÃO NA BASE DE DADOS

In [15]:
df['tweet_text'] = df['tweet_text'].apply(preprocessamento)
#esta linha de código vai rodar por aproximadamente 10min

In [16]:
df.head()

,tweet_text,sentiment
0,o pessoa certo parir seriar o valer e azeved...,1
1,parir mim precisamente o contrário :) vir o ...,1
2,video hoje ... pensar falar sobrar o meter csg...,1
3,aaaaaaaa amar polaroids expressar o quantum to...,1
4,valorizar o coração menino vc diferente o sorr...,1


In [17]:
#localizados valores nulos na coluna tweet text
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   tweet_text  50000 non-null  object
 1   sentiment   50000 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 781.4+ KB


In [18]:
#eliminar os valores nulos
df = df.dropna()

In [19]:
#Linear Support Vector Classification.
#toma como entrada dois arrays: um array X de shape contendo as amostras de treinamento e um array de rótulos de classe (strings ou inteiros)
classifier = LinearSVC()

COMPREENDENDO TF-IDF (TERMO FREQUÊNCIA-FREQUÊNCIA INVERSA DE DOCUMENTO)

TF-IDF significa Frequency Inverse Document Frequency of records. Pode ser definido como o cálculo da relevância de uma palavra em uma série ou corpus para um texto. O significado aumenta proporcionalmente ao número de vezes que uma palavra aparece no texto, mas é compensado pela frequência da palavra no corpus (conjunto de dados).

In [20]:
#Converte uma coleção de documentos brutos em uma matriz de recursos TF-IDF.
tfvectorizer = TfidfVectorizer()

In [21]:
# Features and Labels
xlabels = df['tweet_text']
ylabels = df['sentiment']

In [22]:
#trainning the model
# random state controla o embaralhamento dos dados antes da divisão
X_train, X_test, y_train, y_test = train_test_split(xlabels, ylabels, test_size=0.3, random_state=42)

PIPELINE

Aplica sequencialmente uma lista de transformações e um estimador final. As etapas intermediárias do pipeline devem ser 'transformações', ou seja, devem implementar fit e transform métodos. O estimador final só precisa implementar fit. O objetivo do pipeline é reunir várias etapas que podem ser validadas de forma cruzada ao definir parâmetros diferentes

In [23]:
# Create the  pipeline to vectorize and classify 
pipe = Pipeline([('vectorizer', tfvectorizer),
                 ('classifier', classifier)])

In [24]:
# Fit our data
pipe.fit(X_train,y_train)

Pipeline(steps=[('vectorizer', TfidfVectorizer()), ('classifier', LinearSVC())])

In [25]:
# Predicting with a train dataset
sample_prediction = pipe.predict(X_train)

In [26]:
# Prediction Results
# 1 = Positive review
# 0 = Negative review
for (tweet,pred) in zip(X_train,sample_prediction):
    print(tweet,"Prediction=",pred)

Streaming output truncated to the last 5000 lines.
parir aturar hoje sortir :) Prediction= 1
ai manir esperançar :(( dia bostar Prediction= 0
provir cardiologia pra provar o quantum merda estudar o semana :) Prediction= 1
  o nao abandonar :) Prediction= 0
vir garoto hj e pessoalmente o caro d :) Prediction= 0
ano achar umar foto o site uol deus achar parar achar foto d dia ♥ querer salvar o foto :( rock in rir dia 18/09 Prediction= 0
  anjo :( Prediction= 0
ficar triste o pessoal portar e :((( Prediction= 0
  migar :( Prediction= 0
  opaa valeuu ❤ ️ 1º flamenguista q :) Prediction= 1
  conseguir :( Prediction= 0
pôr o felicidade frente e admirar tar o merda :) Prediction= 1
tou o sair d casar agr :) Prediction= 1
seguir :) Prediction= 1
bar :( deee8dd0-ce0a-413d-a16e-9ab432225f35 Prediction= 0
  q visitar ficar feliz conhecer oq vc n semana vibe e casar voltar esperar coração falar comigo :) Prediction= 1
ai angústia :(( Prediction= 0
precisar conversar o madu :((( Prediction= 0
.....

In [27]:
# Accuracy treino teste e previsão
# Na classificação, essa função calcula a precisão do subconjunto
print("Accuracy: ",pipe.score(X_train,y_train))
print("Accuracy: ",pipe.score(X_test,y_test))
print("Accuracy: ",pipe.score(X_train,sample_prediction))

Accuracy:  0.8701428571428571
Accuracy:  0.724
Accuracy:  1.0


In [ ]:
# Testar com uma frase
frase = input("Escreva um tweet!")
pipe.predict([frase])